In [ ]:
#確認文章id(爬取留言)
#確認留言id(爬取留言回覆)
# 範例:https://api.today.line.me/webapi/comment/list?articleId=74734262&limit=50(顯示筆數)&country=TW&replyCount=true&direction=DESC&postType=&sort=POPULAR&pivot=50(從第幾筆開始)//留言
#即使只有一筆也沒關係，能跑
#第一頁
# 範例:https://api.today.line.me/webapi/comment/list?articleId=74734262&limit=50&country=TW&parentCommentSn=100000010 //留言的回覆

In [ ]:
import pandas as pd
import json
import requests
import time
from bs4 import BeautifulSoup

In [ ]:
new_data=pd.read_csv("/Users/961lin/Desktop/weather/linetoday/text_linetoday_new.csv")
new_data.drop_duplicates(subset=None, keep='first', inplace=True)
new_data.reset_index(inplace=True)

In [ ]:
data_comment=[]
data_reporter=[]
data_time1=[]
data_title=[]
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
for j in range(0,len(new_data)):
    resp=requests.get(new_data['url'][j])
    soup=BeautifulSoup(resp.text,'html5lib')
    items=soup.find_all('script')
    article_id=str(items[1]).split(';')[2].split('"')[1]
#找出文章的留言數量
#先寫一個抓留言的
#(留言數量/30)+1=執行次數
    comment_count=0
    option=True
    a_page_count=0
    while option:
        url='https://api.today.line.me/webapi/comment/list?articleId=%s&limit=30&country=TW&replyCount=true&direction=DESC&postType=&sort=POPULAR&pivot=%s' % (str(article_id),str(a_page_count*30))
        resp=requests.get(url,headers=headers)
        time.sleep(0.5)
        if resp.json()['result']['comments']['comments']!=[]:
            a_page_count+=1
        #number_comment=resp.json()['result']['comments']['count']#留言數量
        #print("此文章留言數量共: %d 則" % number_comment)
            for i in range(0,30):
                try:
                    num1=resp.json()['result']['comments']['comments'][i]['ext']['replyCount']#留言回覆數量
                    comment_id=resp.json()['result']['comments']['comments'][i]['commentSn']
                except:
                    break
                try:
                    reporter=resp.json()['result']['comments']['comments'][i]['displayName']#留言作者
                    print("留言文章:%s" % new_data['title'][j])
                    print("留言作者:%s" % reporter)
                    data_reporter.append(reporter)
                    data_title.append(new_data['title'][j])
                except:
                    data_title.append("None")
                    data_reporter.append("None")
                try:
                    comment_time1=resp.json()['result']['comments']['comments'][i]['createdDate']#時間
                    print("留言時間:%d" % comment_time1)
                    data_time1.append(comment_time1)
                except:
                    data_time1.append("None")
                try:
                    comment=resp.json()['result']['comments']['comments'][i]['contents'][0]['extData']['content']#留言
                    print("留言內容:%s" % comment)
                    data_comment.append(comment)
                    comment_count+=1
                    print("已爬取留言數: %d 則" % comment_count)
                    print("--------------")   
                except:
                    data_comment.append("None")
                    print("****該文章留言已爬取完成*****")
                try:
                    comment_url='https://api.today.line.me/webapi/comment/list?articleId=%s&limit=100&country=TW&parentCommentSn=%s' %(str(article_id),comment_id) 
                    resp1=requests.get(comment_url,headers=headers)
                    comment_id_comment=resp1.json()
                    com_com_count=0
                    print("總共有%d 則回覆" % num1)
                    for k in range(0,num1):

                        if num1>0:
                            try:
                                reporter=comment_id_comment['result']['comments']['comments'][k]['displayName']
                                print("留言文章:%s" % new_data['title'][j])
                                print("留言作者:%s" % reporter)
                                data_reporter.append(reporter)
                                data_title.append(new_data['title'][j])
                            except:
                                data_title.append("None")
                                data_reporter.append("None")
                            try:
                                comment_time1=comment_id_comment['result']['comments']['comments'][k]['createdDate']#時間
                                print("留言時間:%d" % comment_time1)
                                data_time1.append(comment_time1)
                            except:
                                data_time1.append("None")
                            try:
                                comment=comment_id_comment['result']['comments']['comments'][k]['contents'][0]['extData']['content']#留言
                                print("留言內容:%s" % comment)
                                data_comment.append(comment)
                                com_com_count+=1
                                print("已爬取回覆的留言數: %d 則" % com_com_count)
                                print("--------------")
                            except:
                                data_comment.append("None")
                                com_com_count+=1
                                print("------")
                        else:
                            print("該留言回覆不用爬取")
                except:
                    data_reporter.append("None")
                    data_time1.append("None")
                    data_comment.append("None")

        else:
            option=False

In [ ]:
data_commen=pd.DataFrame()
data_commen['title']=data_title
data_commen['reporter']=data_reporter
data_commen['time']=data_time1
data_commen['comment']=data_comment
#data_commen.to_csv("comment_linetoday.csv",index=False,sep=',')